In [111]:
"""
The helper file
"""

import os
import pickle
import copy
import numpy as np


CODES = {'<unk>': 0, '<s>': 1, '</s>': 2}
# Extra vocabulary symbols
sos = '<s>'
eos = '</s>' # also function as PAD
unk = '<unk>'

extra_tokens = [unk, sos, eos]

start_token = extra_tokens.index(sos) # start_token = 0
end_token = extra_tokens.index(eos)   # end_token = 1
unk_token = extra_tokens.index(unk)

def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8') as f:
        data = f.read()

    return data

def preprocess_and_save_data(source_path, target_path):
    """
    Preprocess Text Data.  Save to to file.
    """
    
    # Preprocess
    source_text = load_data(source_path)
    target_text = load_data(target_path)

    source_text = source_text.lower()
    target_text = target_text.lower()

    source_vocab_to_int, source_int_to_vocab = create_lookup_tables(source_text)
    
    target_vocab_to_int, target_int_to_vocab = create_lookup_tables(target_text)

    print(target_int_to_vocab)
    
    source_text, target_text = text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int)

    # Save Data
    pickle.dump((
        (source_text, target_text),
        (source_vocab_to_int, target_vocab_to_int),
        (source_int_to_vocab, target_int_to_vocab)), open('preprocess.p', 'wb'))


def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    """
    vocab = set(text.split())
    vocab_to_int = copy.copy(CODES)
    
    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab


def save_params(params):
    """
    Save parameters to file
    """
    pickle.dump(params, open('params.p', 'wb'))


def load_params():
    """
    Load parameters from file
    """
    return pickle.load(open('params.p', mode='rb'))


def batch_data(source, target, batch_size):
    """
    Batch source and target together
    """
    for batch_i in range(0, len(source)//batch_size):
        start_i = batch_i * batch_size
        source_batch = source[start_i:start_i + batch_size]
        target_batch = target[start_i:start_i + batch_size]
        yield np.array(pad_sentence_batch(source_batch)), np.array(pad_sentence_batch(target_batch))

def pad_sentence_batch(sentence_batch):
    """
    Pad sentence with </s> id
    """
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [CODES['</s>']] * (max_sentence - len(sentence))
            for sentence in sentence_batch]

def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_text_to_id = [[source_vocab_to_int[word] for word in line.split()] for line in source_text.split('\n')]
    target_text_to_id = [[target_vocab_to_int[word] for word in line.split()] for line in target_text.split('\n')]
    
    return (source_text_to_id, target_text_to_id)

In [112]:
import helper
import problem_unittests as tests
import tensorflow as tf

source_path = 'data/small_vocab_fr'
target_path = 'data/small_vocab_en'
source_text = load_data(source_path)
target_text = load_data(target_path)

In [113]:
preprocess_and_save_data(source_path, target_path)

{0: '<unk>', 1: '<s>', 2: '</s>', 3: 'france', 4: 'red', 5: 'mouse', 6: 'strawberries.', 7: 'least', 8: 'wants', 9: 'driving', 10: 'fun', 11: 'she', 12: 'february', 13: 'bananas', 14: 'lemon.', 15: 'strawberries', 16: 'apples', 17: 'apples.', 18: 'school', 19: 'feared', 20: 'peaches', 21: 'fruit.', 22: 'and', 23: 'sometimes', 24: 'india', 25: 'wet', 26: 'favorite.', 27: 'mice', 28: 'fruit', 29: 'mangoes', 30: 'spanish', 31: 'hot', 32: 'march', 33: 'bird', 34: 'black', 35: 'old', 36: 'dogs', 37: 'i', 38: 'chilly', 39: "isn't", 40: 'plan', 41: 'strawberry.', 42: 'lake', 43: 'her', 44: 'mangoes.', 45: 'orange.', 46: 'grape', 47: 'truck', 48: 'would', 49: 'french', 50: 'has', 51: 'want', 52: 'china', 53: 'was', 54: 'am', 55: 'mango.', 56: ',', 57: '?', 58: "it's", 59: 'thinks', 60: 'like', 61: 'bear', 62: 'lemon', 63: 'loved', 64: 'fall', 65: 'lemons.', 66: 'orange', 67: 'oranges.', 68: 'last', 69: 'horse', 70: 'easy', 71: 'october', 72: 'animals', 73: 'why', 74: 'do', 75: 'rainy', 76: 'to

In [114]:
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
# pad_sentence_batch(source_int_text)
source_vocab = len(source_vocab_to_int)
target_vocab = len(target_vocab_to_int)

In [115]:
source_int_text

[[49, 55, 306, 160, 168, 313, 61, 202, 281, 77, 307, 306, 209, 91, 346, 298],
 [262, 289, 306, 299, 105, 91, 173, 281, 77, 307, 127, 26, 91, 301, 298],
 [29, 306, 299, 168, 91, 100, 281, 77, 307, 306, 299, 85, 91, 227, 298],
 [262, 289, 306, 160, 337, 91, 227, 281, 77, 307, 327, 105, 91, 104, 298],
 [117, 250, 37, 14, 306, 58, 342, 281, 255, 97, 250, 37, 306, 54, 24, 298],
 [60, 14, 344, 306, 224, 281, 255, 97, 344, 306, 58, 342, 298],
 [349, 306, 142, 91, 131, 281, 255, 307, 306, 299, 105, 91, 173, 298],
 [49, 55, 306, 71, 42, 290, 281, 77, 307, 306, 283, 47, 91, 100, 298],
 [334, 14, 306, 250, 37, 58, 228, 281, 255, 97, 250, 37, 306, 58, 342, 298],
 [262, 289, 306, 160, 71, 91, 136, 281, 77, 307, 306, 160, 85, 91, 301, 298],
 [54, 215, 306, 60, 250, 37, 325, 274, 281, 255, 54, 130, 306, 97, 250, 10],
 [307, 332, 4, 212, 27, 118, 175, 298],
 [257, 306, 139, 91, 227, 281, 77, 307, 306, 160, 85, 91, 301, 298],
 [15, 200, 191, 97, 239, 58, 278, 37, 298],
 [307, 76, 324, 281, 259, 73, 77,

In [212]:
hidden_units = 256
n_layers = 1
num_encoder_symbols = source_vocab #source vocabulary size
num_decoder_symbols = target_vocab #target vocabulary size
learning_rate = 0.01
embedding_size = 15  #num of features for each word
max_gradient_norm = 5.0
dtype = tf.float32
# Number of Epochs
epochs = 1
# Batch Size
batch_size_train = 256
# Dropout Probability
dropout = 0.2
forget_bias = 1.0
use_beam_search = True
beam_width = 10
length_penalty_weight = 0.0
use_attention = True
learning_rate_decay = False
encoder_type = "bidirectional"

num_encoder_symbols, num_decoder_symbols

(357, 230)

In [213]:
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.contrib.rnn import MultiRNNCell
from tensorflow import layers

tf.reset_default_graph()

train_graph = tf.Graph()
with train_graph.as_default():
    
    ### DEFINING PLACEHOLDERS ###

    # encoder_inputs: [batch_size, max_time_steps]
    encoder_inputs = tf.placeholder(dtype=tf.int32,
                shape=(None, None), name='encoder_inputs')

    # encoder_inputs_length: [batch_size]
    encoder_inputs_length = tf.placeholder(
                dtype=tf.int32, shape=(None,), name='encoder_inputs_length')

    # get dynamic batch_size
    batch_size = tf.shape(encoder_inputs)[0]

    ### TRAIN MODE PLACEHOLDERS ###

    # decoder_inputs: [batch_size, max_time_steps]
    decoder_inputs = tf.placeholder(
                    dtype=tf.int32, shape=(None, None), name='decoder_inputs')

    # decoder_inputs_length: [batch_size]
    decoder_inputs_length = tf.placeholder(
                    dtype=tf.int32, shape=(None,), name='decoder_inputs_length')

    decoder_start_token = tf.ones(
                    shape=[batch_size, 1], dtype=tf.int32) * start_token
    decoder_end_token = tf.ones(
                    shape=[batch_size, 1], dtype=tf.int32) * end_token  


    # decoder_inputs_train: [batch_size , max_time_steps + 1]
    # insert sos symbol in front of each decoder input
    decoder_inputs_train = tf.concat([decoder_start_token,
                                          decoder_inputs], axis=1)

    # decoder_inputs_length_train: [batch_size]
    decoder_inputs_length_train = decoder_inputs_length + 1

    # decoder_targets_train: [batch_size, max_time_steps + 1]
    # insert eos symbol at the end of each decoder input
    decoder_targets_train = tf.concat([decoder_inputs,
                                           decoder_end_token], axis=1)

In [214]:
with train_graph.as_default():
    ## DEFINING ENCODER ##

    encoder_embeddings = tf.Variable(tf.random_uniform([num_encoder_symbols, embedding_size], -1.0, 1.0), dtype=dtype)

    # Embedded_inputs: [batch_size, time_step, embedding_size]
    encoder_inputs_embedded = tf.nn.embedding_lookup(
        params=encoder_embeddings, ids=encoder_inputs)
        
    # Build RNN cell
    cells = []
    for _ in range(n_layers):
        cell = tf.contrib.rnn.BasicLSTMCell(hidden_units, forget_bias=forget_bias)
        if dropout > 0.0:
            cell = tf.contrib.rnn.DropoutWrapper(
                cell=cell, input_keep_prob=(1.0 - dropout))
        cells.append(cell)
    if n_layers == 1:
        encoder_cells = cells[0]
    else:
        encoder_cells = tf.contrib.rnn.MultiRNNCell(cells)

    if encoder_type == "bidirectional":
        
        num_bi_layers = int(n_layers / 2)
        num_residual_layers = n_layers - 1
        num_bi_residual_layers = int(num_residual_layers / 2)
        
        cell_list = []
        for i in range(n_layers):
            cell = tf.contrib.rnn.BasicLSTMCell(hidden_units, forget_bias=forget_bias)

            if (i >= n_layers - num_residual_layers):
                cell = tf.contrib.rnn.ResidualWrapper(cell, residual_fn=None)
                if dropout > 0.0:
                    cell = tf.contrib.rnn.DropoutWrapper(
                        cell=cell, input_keep_prob=(1.0 - dropout))
            
            cell_list.append(cell)
            
        if len(cell_list) == 1:  # Single layer.
            fw_cell = cell_list[0]
            bw_cell = cell_list[0]
        else:  # Multi layers
            fw_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
            bw_cell = tf.contrib.rnn.MultiRNNCell(cell_list)

        bi_outputs, bi_state = tf.nn.bidirectional_dynamic_rnn(
                                                        fw_cell,
                                                        bw_cell,
                                                        encoder_inputs_embedded,
                                                        dtype=dtype,
                                                        sequence_length=encoder_inputs_length,
                                                        time_major=False,
                                                        swap_memory=True)

        encoder_outputs, bi_encoder_state = tf.concat(bi_outputs, -1), bi_state
        
        if num_bi_layers == 1:
            encoder_last_state = bi_encoder_state
        else:
            # alternatively concat forward and backward states
            encoder_state = []
            for layer_id in range(num_bi_layers):
                encoder_state.append(bi_encoder_state[0][layer_id])  # forward
                encoder_state.append(bi_encoder_state[1][layer_id])  # backward
            encoder_last_state = tuple(encoder_state)
        
    else:
        encoder_outputs, encoder_last_state = tf.nn.dynamic_rnn(
            cell=encoder_cells, inputs=encoder_inputs_embedded,
            sequence_length=encoder_inputs_length, dtype=dtype,
            time_major=False)

In [215]:
with train_graph.as_default():
    ### DEFINING DECODER ###

    
    # Building decoder_cell
    cells = []
    # Build RNN cell
    for _ in range(n_layers):
        cell = tf.contrib.rnn.BasicLSTMCell(hidden_units, forget_bias=forget_bias)
        if dropout > 0.0:
            cell = tf.contrib.rnn.DropoutWrapper(
                cell=cell, input_keep_prob=(1.0 - dropout))
        cells.append(cell)
    if n_layers == 1:
        decoder_cells = cells[0]
    else:
        decoder_cells = tf.contrib.rnn.MultiRNNCell(cells)

    if use_attention:
        memory = encoder_outputs
        
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
            hidden_units,
            memory,
            memory_sequence_length=encoder_inputs_length,
            normalize=True)
        
        decoder_cells_train = tf.contrib.seq2seq.AttentionWrapper(
            decoder_cells,
            attention_mechanism,
            attention_layer_size=hidden_units,
            alignment_history=False,
            output_attention=True,
            name="attention")
        
        decoder_initial_state = decoder_cells_train.zero_state(batch_size, dtype).clone(
          cell_state=encoder_last_state)
        
    else:
        decoder_cells_train = decoder_cells
        decoder_initial_state = encoder_last_state

    decoder_embeddings = tf.Variable(tf.random_uniform([num_decoder_symbols, embedding_size], -1.0, 1.0), dtype=dtype)
    
    # decoder_inputs_embedded: [batch_size, max_time_step + 1, embedding_size]
    decoder_inputs_embedded = tf.nn.embedding_lookup(
        params=decoder_embeddings, ids=decoder_inputs_train)

In [216]:
with train_graph.as_default():
    ### TRAIN MODE ###
    
    # Helper to feed inputs for training: read inputs from dense ground truth vectors
    training_helper = seq2seq.TrainingHelper(inputs=decoder_inputs_embedded,
                                       sequence_length=decoder_inputs_length_train,
                                       time_major=False,
                                        name='training_helper')

    training_decoder = seq2seq.BasicDecoder(cell=decoder_cells_train,
                                       helper=training_helper,
                                       initial_state=decoder_initial_state)

    # decoder_outputs_train: BasicDecoderOutput
    #                        namedtuple(rnn_outputs, sample_id)
    # decoder_outputs_train.rnn_output: [batch_size, max_time_step + 1, num_decoder_symbols] if output_time_major=False
    #                                   [max_time_step + 1, batch_size, num_decoder_symbols] if output_time_major=True
    # decoder_outputs_train.sample_id: [batch_size], tf.int32
    (decoder_outputs_train, decoder_last_state_train, 
         decoder_outputs_length_decode)  = seq2seq.dynamic_decode(decoder=training_decoder,
                                                        output_time_major=False,
                                                        swap_memory=True,
                                                        impute_finished=True)

    # More efficient to do the projection on the batch-time-concatenated tensor
    # logits_train: [batch_size, max_time_step + 1, num_decoder_symbols]
    
    sample_id = decoder_outputs_train.sample_id
    
    output_layer = layers.Dense(num_decoder_symbols, name='output_projection')
    logits_train = output_layer(decoder_outputs_train.rnn_output)

ValueError: not enough values to unpack (expected 2, got 0)

In [93]:
with train_graph.as_default():
    
    ### LOSS, GRADIEND AND OPTIMIZATION ###
    
    if learning_rate_decay:
        global_step = tf.Variable(0, trainable=False)

        learning_rate = tf.constant(learning_rate)

        #using luong10 decay scheme
        decay_factor = 0.5
        start_decay_step = int(epochs / 2)
        decay_times = 10

        remain_steps = epochs - start_decay_step
        decay_steps = int(remain_steps / decay_times)

        learning_rate = tf.cond(global_step < start_decay_step,
                                lambda: learning_rate,
                                lambda: tf.train.exponential_decay(
                                    learning_rate,
                                    (global_step - start_decay_step),
                                    decay_steps, decay_factor, staircase=True),
                                name="learning_rate_decay_cond")
    
    # Maximum decoder time_steps in current batch
    max_decoder_length = tf.reduce_max(decoder_inputs_length_train)
    
    # masks: masking for valid and padded time steps, [batch_size, max_time_step + 1]
    target_weights = tf.sequence_mask(lengths=decoder_inputs_length_train, 
                             maxlen=max_decoder_length, dtype=dtype, name='masks')
    
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=decoder_targets_train, logits=logits_train)
    
    loss = (tf.reduce_sum(crossent * target_weights) /
        tf.cast(batch_size, dtype= dtype))

    trainable_params = tf.trainable_variables()
    
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    gradients = tf.gradients(loss, 
                             trainable_params)
    
    clip_gradients, gradient_norm = tf.clip_by_global_norm(gradients, max_gradient_norm)
    
    updates = opt.apply_gradients(
            zip(clip_gradients, trainable_params))
    

ValueError: Tensor("output_projection/BiasAdd:0", shape=(?, ?, 230), dtype=float32) must be from the same graph as Tensor("concat_1:0", shape=(?, ?), dtype=int32).

In [87]:
with train_graph.as_default():

    ### INFERENCE MODE ###
    
    start_tokens = tf.fill([batch_size], start_token)
    
    if use_attention:
        memory = tf.contrib.seq2seq.tile_batch(
          memory, multiplier=beam_width)
        
        source_sequence_length = tf.contrib.seq2seq.tile_batch(
          encoder_inputs_length, multiplier=beam_width)
        
        encoder_last_state = tf.contrib.seq2seq.tile_batch(
          encoder_last_state, multiplier=beam_width)
        
        batch_size = batch_size * beam_width
        
        attention_mechanism_infer = tf.contrib.seq2seq.BahdanauAttention(
            hidden_units,
            memory,
            memory_sequence_length=source_sequence_length,
            normalize=True)
        
        decoder_cells_infer = tf.contrib.seq2seq.AttentionWrapper(
            decoder_cells,
            attention_mechanism_infer,
            attention_layer_size=hidden_units,
            alignment_history=False,
            output_attention=True,
            name="attention_infer")
        
        decoder_initial_state_infer = decoder_cells_infer.zero_state(batch_size, dtype).clone(
          cell_state=encoder_last_state)
    
    if use_beam_search:
        #     decoder_initial_state_infer = tf.contrib.seq2seq.tile_batch(
        #           encoder_last_state, multiplier=beam_width)

        inference_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
              cell=decoder_cells_infer,
              embedding=decoder_embeddings,
              start_tokens=start_tokens,
              end_token=end_token,
              initial_state=decoder_initial_state_infer,
              beam_width=beam_width,
              output_layer=output_layer,
              length_penalty_weight=length_penalty_weight)
        
    else:
        inference_helper = seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                        start_tokens=start_tokens,
                                                        end_token=end_token)

        inference_decoder = seq2seq.BasicDecoder(cell=decoder_cells_infer,
                                                 helper=inference_helper,
                                                 initial_state=decoder_initial_state,
                                                 output_layer=output_layer)
    

    
    maximum_iterations = tf.round(tf.reduce_max(encoder_inputs_length) * 2)
    
    (decoder_infer_outputs, decoder_infer_last_state,
                 decoder_infer_outputs_length) = (seq2seq.dynamic_decode(
                    decoder=inference_decoder,
                    output_time_major=False,
                    maximum_iterations=maximum_iterations))
    
    if use_beam_search:
        decoder_pred_decode = decoder_infer_outputs.predicted_ids
        tf.identity(decoder_pred_decode, 'decoder_pred_decode')
    
    else:
        logits_infer = decoder_infer_outputs.rnn_output
        sample_id_infer = decoder_infer_outputs.sample_id

In [63]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    lower_case_words = [word.lower() for word in sentence.split()]
    
    word_id = [vocab_to_int.get(word, vocab_to_int['<unk>']) for word in lower_case_words]
    
    return word_id

import time

### TRAINING ###
save_path = 'checkpoints/dev'

train_source = source_int_text[:]
train_target = target_int_text[:]

valid_source = source_int_text[:batch_size_train]
valid_target = target_int_text[:batch_size_train]

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                batch_data(train_source, train_target, batch_size_train)):
            start_time = time.time()
            
            source_batch_seq_lenght = []
            for item in source_batch:
                source_batch_seq_lenght.append(np.shape(item)[0])
            
            target_batch_seq_lenght = []
            for item in target_batch:
                target_batch_seq_lenght.append(np.shape(item)[0])
                
            print(source_batch_seq_lenght)
            print(target_batch_seq_lenght)
            
            if (source_batch_seq_lenght[0] > 300):
                continue
                
            _, loss_val = sess.run(
                [updates, loss],
                {encoder_inputs: source_batch,
                 decoder_inputs: target_batch,
                encoder_inputs_length: source_batch_seq_lenght,
                decoder_inputs_length: target_batch_seq_lenght})

            print('Epoch {:>3} Batch {:>4}/{}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size_train, loss_val))
            
            end_time = time.time()
            
    print("Training time: ", end_time - start_time)
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,

Epoch   0 Batch    4/538, Loss: 136.011
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,

Epoch   0 Batch   10/538, Loss: 58.431
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 

Epoch   0 Batch   16/538, Loss: 59.237
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 

Epoch   0 Batch   22/538, Loss: 41.354
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 

...

Epoch   0 Batch  512/538, Loss:  0.982
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 

Epoch   0 Batch  518/538, Loss:  0.747
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 

Epoch   0 Batch  525/538, Loss:  0.716
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 

Epoch   0 Batch  532/538, Loss:  0.862
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 

Epoch   0 Batch  536/538, Loss:  0.898
[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 

In [64]:
# Save parameters for checkpoint
save_params(save_path)

In [65]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()
load_path = load_params()

In [67]:
translate_sentence = "new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."
#fr to en
#input: "new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."
#target:"new jersey is sometimes quiet during autumn , and it is snowy in april ."

#en to vi
#input:  "Họ viết gần 1000 trang về chủ đề này ."
#target: "They wrote almost a thousand pages on the topic ."

print(translate_sentence)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)
print(np.shape(translate_sentence))

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    encoder_inputs = loaded_graph.get_tensor_by_name('encoder_inputs:0')
    encoder_inputs_length = loaded_graph.get_tensor_by_name('encoder_inputs_length:0')
    decoder_pred_decode = loaded_graph.get_tensor_by_name('decoder_pred_decode:0')
    
    predicted_ids = sess.run(decoder_pred_decode, {encoder_inputs: [translate_sentence],
                                                       encoder_inputs_length: [np.shape(translate_sentence)[0]]})[0]
    
    print(predicted_ids)
    

# with tf.Session(graph=train_graph) as sess:
#     sess.run(tf.global_variables_initializer())
    
#     translate_logits = sess.run(decoder_pred_decode, {encoder_inputs: [translate_sentence],
#                                                        encoder_inputs_length: [np.shape(translate_sentence)[0]]})[0]
    
#     print(translate_logits)

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  Source Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i[0] for i in predicted_ids]))
print('  Predicted Words: {}'.format([target_int_to_vocab[i[0]] for i in predicted_ids]))

print('\nTranslation:\n')
translation = ''
for word_i in translate_logits:
    translation += target_int_to_vocab[word_i[0]] + ' '
    
print(translation)

new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
(16,)
INFO:tensorflow:Restoring parameters from checkpoints/dev
[[223 223 223 223 223 223 223 223 223 223]
 [146 146 146 146 146 146 146 146 146 146]
 [146 146 146 146 146 146 146 146 146 146]
 [181 181 181 181 181 181 181 181 181 181]
 [179 179 179 179 179 179 179 179 179 179]
 [ 76  76  76  76  76  76  76  76  76  76]
 [ 58  58  58  58  58  58  58  58  58  58]
 [ 30  30  30  30  30  30  30  30  30  30]
 [154 154 154 154 154 154 154 154 154 154]
 [ 88  88  88  88  88  88  88  88  88  88]
 [154 154 154 154 154 154 154 154 154 154]
 [ 88  88  88  88  88  88  88  88  88  88]
 [154 154 154 154 154 154 154 154 154 154]
 [ 70  70  70  70  70  70  70  70  70  70]
 [154 154 154 154 154 154 154 154 154 154]
 [ 88  88  88  88  88  88  88  88  88  88]
 [ 70  70  70  70  70  70  70  70  70  70]
 [ 88  88  88  88  88  88  88  88  88  88]
 [ 70  70  70  70  70  70  70  70  70  70]
 [ 70  70  70  70  70  70  70  70  70  7